In [ ]:
# Nueva importación modularizada y simplificada
from src.preprocessing import load_data, clean_data, feature_engineering, scale_and_split

In [ ]:
data_path = "../data/raw/sensor_data.csv"
df = load_data(data_path)  # Usa función modularizada

,timestamp,temperatura,vibracion,presion,falla
0,2024-01-01 00:00:00,72.48,2.79,28.64,0
1,2024-01-01 01:00:00,69.31,2.77,29.39,0
2,2024-01-01 02:00:00,73.24,2.10,28.81,0
3,2024-01-01 03:00:00,77.62,2.83,30.22,0
4,2024-01-01 04:00:00,68.83,3.37,32.39,0


In [4]:
# Comprobación inicial de valores faltantes
faltantes = df.isnull().sum().sum()

# Tratamiento condicional según existencia de datos faltantes
if faltantes > 0:
    # Eliminar filas con datos faltantes (alternativamente se podría imputar)
    df = df.dropna().reset_index(drop=True)
    print(f"✅ Se eliminaron filas con datos faltantes. Filas restantes: {len(df)}")
else:
    print("✅ Confirmado: No existen datos faltantes en el dataset.")


✅ Confirmado: No existen datos faltantes en el dataset.


In [5]:
# Convertimos columna timestamp al formato datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Creamos nuevas características temporales a partir del timestamp
df['hora'] = df['timestamp'].dt.hour             # Extraemos la hora del día
df['dia_semana'] = df['timestamp'].dt.dayofweek  # Día de la semana (0: lunes, 6: domingo)

# Eliminamos columna timestamp original (no útil directamente para el modelo)
df = df.drop(columns=['timestamp'])

# Verificamos la nueva estructura del dataset
df.head()


,temperatura,vibracion,presion,falla,hora,dia_semana
0,72.48,2.79,28.64,0,0,0
1,69.31,2.77,29.39,0,1,0
2,73.24,2.10,28.81,0,2,0
3,77.62,2.83,30.22,0,3,0
4,68.83,3.37,32.39,0,4,0


In [6]:
# Variable objetivo: detección de fallas
y = df['falla']

# Variables predictoras: todas excepto 'falla'
X = df.drop(columns=['falla'])

# Confirmamos dimensiones de los datasets
print(f"Tamaño de características (X): {X.shape}")
print(f"Tamaño de variable objetivo (y): {y.shape}")

Tamaño de características (X): (5000, 5)
Tamaño de variable objetivo (y): (5000,)


In [7]:
# División de los datos en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,                    # 20% para prueba
    random_state=42,                  # Asegura reproducibilidad
    stratify=y                        # Distribución proporcional de la variable objetivo
)

# Verificación de tamaño
print("Conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Conjunto de prueba:", X_test.shape, y_test.shape)


Conjunto de entrenamiento: (4000, 5) (4000,)
Conjunto de prueba: (1000, 5) (1000,)


In [ ]:
# Instancia del escalador estándar
scaler = StandardScaler()

# Ajuste del escalador solo en datos de entrenamiento y transformación
X_train_scaled = scaler.fit_transform(X_train)

# Aplicar la transformación ajustada al conjunto de prueba
X_test_scaled = scaler.transform(X_test)

# Convertimos nuevamente en DataFrames por comodidad
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Mostramos un ejemplo de cómo quedaron escalados los datos
X_train_scaled.head()

,temperatura,vibracion,presion,hora,dia_semana
0,0.185706,-0.731048,0.249640,1.230223,1.014544
1,-1.169397,-0.174283,-1.091506,-0.217056,0.514768
2,0.738605,-0.194167,0.927692,0.361856,-0.984557
3,0.274170,-1.148623,0.783107,0.651311,-1.484332
4,0.439034,1.973243,0.837949,-0.072328,-1.484332


In [9]:
# Rutas para guardar los datos procesados
processed_path = "../data/processed/"
os.makedirs(processed_path, exist_ok=True)  # Crea carpeta si no existe

# Guardamos los datos procesados
X_train_scaled.to_csv(processed_path + 'X_train_scaled.csv', index=False)
X_test_scaled.to_csv(processed_path + 'X_test_scaled.csv', index=False)
y_train.to_csv(processed_path + 'y_train.csv', index=False)
y_test.to_csv(processed_path + 'y_test.csv', index=False)

print("✅ Datos procesados guardados correctamente.")

✅ Datos procesados guardados correctamente.
